The goal of this file is to test Cassiopeia and figure out what data I can access via the API.  Some examples are provided on the cassiopeia repository https://github.com/meraki-analytics/cassiopeia

Good practice would be extracting my ARAM match history for this season and storing the a few data points in a dataframe.

Something like:
            Champion   kills    deaths   assists   win   date    patch
match_id

In [1]:
import cassiopeia as cass
from sortedcontainers import SortedList
import arrow
import pandas as pd
import config
import csv
import time

cass.set_riot_api_key(config.API_KEY)
cass.set_default_region('NA')

summoner = cass.get_summoner(name = "StoneMonkey76")

summoner.level

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/StoneMonkey76


308

In [2]:
match_history = cass.core.MatchHistory(summoner = summoner, queues = {cass.Queue.aram})

Match history is a list of Aram matches for me.  Lets find the timeline for these matches as well.

In [3]:
p = match_history[0].participants[summoner]

Making call: https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/klHbpjEWpk00Y33fZd1e00-vSvkAZm20iU8ThiCML6THl3U?beginIndex=0&endIndex=100&queue=450


In [4]:
print(dir(match_history[0].participants[summoner].stats))

Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3947623337
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-account/klHbpjEWpk00Y33fZd1e00-vSvkAZm20iU8ThiCML6THl3U
['_ParticipantStats__match', '_ParticipantStats__participant', '__call__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__json__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data', '_data_types', '_renamed', 'altars_captured', 'altars_neutralized', 'assists', 'combat_player_score', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets', 'damage_self_mitigated', 'deaths', 'double_kills', 'first_blood_assist', 'first_blood_kill', 'first_inhibitor_assist',

In [5]:
p_stats = match_history[0].participants[summoner].stats

In [6]:
print(p.summoner.name, "playing", p.champion.name)
print(p.summoner.name, "score: ",p_stats.kills,'/',p_stats.deaths,'/',p_stats.assists)


Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/11.12.1/data/en_US/championFull.json
StoneMonkey76 playing Taliyah
StoneMonkey76 score:  11 / 6 / 18


In [7]:
print(dir(cass.core.Match))

['_Ghost__all_loaded', '_Ghost__is_loaded', '_Ghost__load_groups', '_Ghost__property', '_Ghost__set_loaded', '__call__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get_query__', '__get_query_from_kwargs__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__json__', '__le__', '__load__', '__load_hook__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_construct_normally', '_data_types', '_load_type', '_load_types', '_renamed', 'blue_team', 'creation', 'duration', 'exists', 'from_data', 'from_match_reference', 'id', 'is_remake', 'kills_heatmap', 'load', 'map', 'mode', 'participants', 'patch', 'platform', 'property', 'queue', 'red_team', 'region', 'season', 'teams', 'timeline', 'to_dict', 'to_json', 'type', 'version']


In [8]:
print(match_history[0].creation.format("YYYY/MM/DD"))
print(match_history[0].patch)

2021/06/18
11.12


Now I will get a list of all of my ARAM games during season 11.  AKA. games beginning at patch 11.1 and ending at now

In [9]:
start_patch = cass.Patch.from_str("11.1").start
end_time = arrow.now()
match_history = cass.core.MatchHistory(summoner = summoner, queues = {cass.Queue.aram}, begin_time = start_patch, end_time = end_time)

In [10]:
len(match_history)

Making call: https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/klHbpjEWpk00Y33fZd1e00-vSvkAZm20iU8ThiCML6THl3U?beginIndex=0&endIndex=100&queue=450
Making call: https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/klHbpjEWpk00Y33fZd1e00-vSvkAZm20iU8ThiCML6THl3U?beginIndex=100&endIndex=200&queue=450


192

This shows that at the moment I have played 161 ARAM games this season.  Now what I need to do is extract the desired statistics from those games and insert them into a data frame.
            Champion   kills    deaths   assists   win   date    patch
match_id

In [11]:
#First create empty frame with column names
df = pd.DataFrame(columns = ['Match_id','Champion','Kills','Deaths','Assists','Win','Date','Patch'])

for match in match_history:
    my_data = match.participants[summoner]
    match_dict = {"Match_id": match.id,
                  "Champion": my_data.champion.name,
                  "Kills" : my_data.stats.kills,
                  "Deaths" : my_data.stats.deaths,
                  "Win" : my_data.stats.win,
                  "Assists" : my_data.stats.assists,
                  "Date" : match.creation.format("YYYY/MM/DD"),
                  "Patch" : match.patch,
                  }
    df = df.append(match_dict, ignore_index = True)

Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946723396
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946668271
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946654292
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946641139
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946547632
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946553602
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946498900
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3946495239
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3945371540
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3942740389
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3942668426
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3942647419
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3942626075
Making call: https://na1.

Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3812035595
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3811747157
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3811649244
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3811029591
Making call: https://ddragon.leagueoflegends.com/cdn/11.4.1/data/en_US/championFull.json
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3809715677
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3808673644
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3807236022
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3807223013
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3807149265
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3807165111
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3803134976
Making call: https://na1.api.riotgames.com/lol/match/v4/matches/3802870769
Making call

In [12]:
df

,Match_id,Champion,Kills,Deaths,Assists,Win,Date,Patch
0,3947623337,Taliyah,11,6,18,True,2021/06/18,11.12
1,3946723396,Rengar,22,14,22,False,2021/06/17,11.12
2,3946668271,Kha'Zix,19,13,23,True,2021/06/17,11.12
3,3946654292,Varus,6,9,24,False,2021/06/17,11.12
4,3946641139,Irelia,14,12,14,False,2021/06/17,11.12
...,...,...,...,...,...,...,...,...
187,3748258871,Irelia,6,19,16,False,2021/01/18,11.1
188,3744426248,Nidalee,9,2,18,True,2021/01/15,11.1
189,3740387700,LeBlanc,19,10,15,True,2021/01/12,11.1
190,3739349273,Kha'Zix,13,9,14,True,2021/01/11,11.1


Now df is a DataFrame of all the ARAM games I have played this season.
Next, I should practice manipulating the data of these ARAM games. It could be interesting to find out things such as Champion by win rate, win rate by patch, and such.  I will move Match_id to the index and see if that is a QOL change or not.

In [13]:
df = df.set_index("Match_id")
df.head()

,Champion,Kills,Deaths,Assists,Win,Date,Patch
Match_id,,,,,,,
3947623337,Taliyah,11,6,18,True,2021/06/18,11.12
3946723396,Rengar,22,14,22,False,2021/06/17,11.12
3946668271,Kha'Zix,19,13,23,True,2021/06/17,11.12
3946654292,Varus,6,9,24,False,2021/06/17,11.12
3946641139,Irelia,14,12,14,False,2021/06/17,11.12


In [14]:
#Champion win rates
champs_played = df["Champion"].unique()
df_win_rate = pd.DataFrame(columns = ['Champion', 'Win Rate', 'Games Played'])

for champ in champs_played:
    df_test = df[df["Champion"] == champ]
    try:
        wr = (len(df_test[df_test['Win']])/len(df_test)) * 100
    except ZeroDivisionError:
        wr = 0
    df_win_rate = df_win_rate.append({"Champion": champ, "Win Rate": wr, "Games Played": len(df_test)}, ignore_index = True)

df_win_rate = df_win_rate.set_index("Champion")

In [15]:
df_win_rate = df_win_rate.sort_values(by= ["Win Rate", "Games Played"], ascending = False)
df_win_rate

,Win Rate,Games Played
Champion,,
Sion,100.0,7
Nasus,100.0,3
Xayah,100.0,3
Rek'Sai,100.0,3
Viktor,100.0,3
...,...,...
Vi,0.0,1
Morgana,0.0,1
Xerath,0.0,1


Useful facts:
Season is depreciated! Use dates and times
Patch 11.1 was the first season 11 patch

Useful calls:

cass.core.Match has: creation(date), patch, id

cass.core.Match.participants.stats has: kills, deaths, assists, champion, win

cass.Data.Queues.ranked_solo_fives

In [16]:
df_win_rate[df_win_rate["Win Rate"] == 0]

,Win Rate,Games Played
Champion,,
Irelia,0.0,4
Diana,0.0,4
Miss Fortune,0.0,2
Kayle,0.0,2
Aatrox,0.0,2
Veigar,0.0,2
Singed,0.0,2
Draven,0.0,2
Malzahar,0.0,2


In [17]:
df_win_rate[df_win_rate['Win Rate'] ==100]

,Win Rate,Games Played
Champion,,
Sion,100.0,7
Nasus,100.0,3
Xayah,100.0,3
Rek'Sai,100.0,3
Viktor,100.0,3
Shyvana,100.0,2
Gwen,100.0,2
Neeko,100.0,2
Gangplank,100.0,1


In [18]:
x = df_win_rate[df_win_rate['Games Played']>=2]
x.sort_values(by='Games Played', ascending = False)

,Win Rate,Games Played
Champion,,
Nidalee,36.363636,11
LeBlanc,50.000000,10
Kindred,44.444444,9
Cassiopeia,62.500000,8
Rengar,57.142857,7
Sion,100.000000,7
Kai'Sa,33.333333,6
Varus,66.666667,6
Jhin,66.666667,6
